# Gradient Tree Boosting

In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from src.gradboost import GradBoost
import src.utils as utils

In [ ]:
df = pd.read_csv("data/diamonds.csv")
X_train, X_test, y_train, y_test = utils.preprocessing_LOS(df)

In [ ]:
param_grid = {
	"loss": ["log_loss", "exponential"],
	"learning_rate": [0.01, 0.1, 0.5, 1.0],
	"subsample": [0.5, 0.75, 1.0],
	"criterion": ["friedman_mse", "squared_error"],
}

grid_search = RandomizedSearchCV(GradBoost(), param_grid, cv=5, scoring="accuracy", 
								 return_train_score=True, verbose=10, n_iter=10, n_jobs=-1)
grid_search.fit(X_train, y_train)